In [ ]:
# install pyscf
# will simplify setup later...
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import gto
from scipy.io import savemat
from utils.buildBreadthFirst3d import buildBreadthFirst3d
from utils.plot3dtree import plot3dtree
from utils.cheby import cheby
from utils.vals2coeffs3d import vals2coeffs3d
from utils.ischeckResolved import ischeckResolved3d

# basis
mol_h2o = gto.M(
    verbose = 0,
    atom = '''
    o    0    0.       0.
    h    0    -0.757   0.587
    h    0    0.757    0.587''',
    # basis = 'ccpvdz')    
    basis = 'ccpvtz')

dom = numpy.array([-5, 5, -5, 5, -5, 5])
tol = 1.0e-4
p = 8
checkpts = numpy.array([[ 0,    0.,       0.],
                        [ 0,   -0.757,    0.587],
                        [ 0,    0.757,    0.587]])
ifcoeffs = True
nd1 = 58 # ccpvtz


In [ ]:
# if want to resolve all N_orb**2 basis
nd = nd1**2
def pyscffunc_all(x, y, z, mol):
  valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
  nrows, ncols = valstmp.shape
  vals = numpy.zeros((nrows, ncols**2))
  for j in range(ncols):
    for k in range(ncols):
      vals[:, j*ncols+k] = valstmp[:, j] * valstmp[:, k]
  return vals
func_all = lambda x, y, z: pyscffunc_all(x, y, z, mol_h2o) # compute func2 on tree3
func = func_all


In [ ]:
# if want to resolve 1 proxy func
nd = 1
def pyscffunc_proxy(x, y, z, mol):
  valstmp = numpy.array(mol.eval_gto('GTOval_sph',numpy.column_stack([x.flatten(),y.flatten(),z.flatten()])))
  nrows, ncols = valstmp.shape
  vals = numpy.zeros((nrows))
  for j in range(ncols):
    vals = vals + valstmp[:, j]**2
  return vals
func_proxy = lambda x, y, z: pyscffunc_proxy(x, y, z, mol_h2o)
func = func_proxy


In [ ]:
# build tree for func, will move initialization to utils
tree3_h2o = {
    'domain': dom.reshape(-1,1), 
    'tol': tol,
    'nSteps': 15,
    'n': p,
    'checkpts': checkpts.transpose(),
    'ifcoeffs': ifcoeffs                           
  }
numpts = 51 # this needs to be consistent with the resolution in plot3dtree.m
tree3_h2o, rint = buildBreadthFirst3d(tree3_h2o, func)
xx, yy, zz = numpy.meshgrid(numpy.linspace(dom[0],dom[1],numpts),numpy.linspace(dom[2],dom[3],numpts),numpy.linspace(dom[4],dom[5],numpts),indexing='ij')
v3 = func(xx.flatten(),yy.flatten(),zz.flatten())
savemat('tree3.mat', {'nd': nd, 'numpts': numpts, 'v': v3, 'xx': xx, 'yy': yy, 'zz': zz, 'rint': rint, 'fdomain': tree3_h2o['domain'], 'fn': tree3_h2o['n'], 'flevel': tree3_h2o['level'], 'fchildren': tree3_h2o['children'], 'fheight': tree3_h2o['height'], 'fid': tree3_h2o['id'], 'frint': tree3_h2o['rint'], 'ftol': tree3_h2o['tol'], 'fcheckpts': tree3_h2o['checkpts']})


In [ ]:
# check error: nd1**2 num of basis to check...
checkfunc = lambda x, y, z: pyscffunc_all(x, y, z, mol_h2o) # compute func2 on tree3
# tree3_h2o['ifcoeffs'] = False
tree3_h2o['ifcoeffs'] = ifcoeffs
tree3_h2o, checkerror = ischeckResolved3d(tree3_h2o, checkfunc, nd1**2)
savemat('tree_error.mat', {'tol': tree3_h2o['tol'], 'fcheckerror': tree3_h2o['checkerror'],'fheight': tree3_h2o['height'],'checkerror': checkerror})
numpy.max(checkerror)